## Part 1: Preprocessing

In [34]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [35]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [36]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

# Preview the new DataFrame
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [37]:
# Create a list of at least 10 column names to use as X data
feature_columns = [
    "Age",
    "BusinessTravel",
    "DistanceFromHome",
    "Education",
    "EnvironmentSatisfaction",
    "JobInvolvement",
    "JobLevel",
    "MaritalStatus",
    "YearsSinceLastPromotion",
    "OverTime"
]

# Create X_df using your selected columns
X_df = attrition_df[feature_columns]

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
BusinessTravel,object
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobInvolvement,int64
JobLevel,int64
MaritalStatus,object
YearsSinceLastPromotion,int64
OverTime,object


In [38]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

# Confirm the split worked
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1102, 10)
(368, 10)
(1102, 2)
(368, 2)


In [39]:
# Convert X_train and X_test to numeric using one-hot encoding
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

# Ensure the same columns exist in both sets
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Preview encoded X_train
X_train_encoded.head()


,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,YearsSinceLastPromotion,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
464,37,1,3,2,3,3,1,False,False,True,False,False,True,True,False
512,27,3,4,1,2,1,1,False,False,True,False,False,True,True,False
480,30,12,4,2,2,1,0,False,True,False,False,True,False,True,False
680,36,7,4,2,3,1,0,False,False,True,False,False,True,True,False
1023,56,1,2,1,3,1,1,False,False,True,False,True,False,True,False


In [40]:
# Count the values in the 'OverTime' column of the training data
X_train["OverTime"].value_counts().to_frame(name="count")


,count
OverTime,
No,798
Yes,304


In [41]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Convert the scaled data back into DataFrames (so we retain the column names)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)

# Preview the scaled training data
X_train_scaled.head()


,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,YearsSinceLastPromotion,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,-0.000597,-1.009337,0.080843,-0.660419,0.373771,0.845308,-0.350760,-0.347934,-0.480921,0.648154,-0.531871,-0.914695,1.444588,0.617213,-0.617213
1,-1.097736,-0.761501,1.070722,-1.578176,-1.013082,-0.963487,-0.350760,-0.347934,-0.480921,0.648154,-0.531871,-0.914695,1.444588,0.617213,-0.617213
2,-0.768595,0.353763,1.070722,-0.660419,-1.013082,-0.963487,-0.663492,-0.347934,2.079344,-1.542844,-0.531871,1.093261,-0.692239,0.617213,-0.617213
3,-0.110311,-0.265828,1.070722,-0.660419,0.373771,-0.963487,-0.663492,-0.347934,-0.480921,0.648154,-0.531871,-0.914695,1.444588,0.617213,-0.617213
4,2.083967,-1.009337,-0.909036,-1.578176,0.373771,-0.963487,-0.350760,-0.347934,-0.480921,0.648154,-0.531871,1.093261,-0.692239,0.617213,-0.617213


In [42]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data (make sure the column is passed as a DataFrame or 2D array)
encoder.fit(y_train[['Department']])

# Apply the encoder to the training and testing data
y_train_department_encoded = encoder.transform(y_train[['Department']])
y_test_department_encoded = encoder.transform(y_test[['Department']])

# Preview the encoded output for the training data
print(y_train_department_encoded)



[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [43]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder
encoder_attrition = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data (ensure 'Attrition' is passed as a DataFrame or 2D array)
encoder_attrition.fit(y_train[['Attrition']])

# Apply the encoder to the training and testing data
y_train_attrition_encoded = encoder_attrition.transform(y_train[['Attrition']])
y_test_attrition_encoded = encoder_attrition.transform(y_test[['Attrition']])

# Preview the encoded output for the training data
print(y_train_attrition_encoded)



[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


## Part 2: Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data.


# Create the input layer


# Create at least two shared layers


In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer


In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer


In [ ]:
# Create the model


# Compile the model


# Summarize the model


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - attrition_output_accuracy: 0.6931 - department_output_accuracy: 0.5956 - loss: 1.4833
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8335 - department_output_accuracy: 0.6383 - loss: 1.2524
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8421 - department_output_accuracy: 0.6393 - loss: 1.1859
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8389 - department_output_accuracy: 0.6692 - loss: 1.1473
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8714 - department_output_accuracy: 0.6484 - loss: 1.0626
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8700 - department_output_accuracy: 0.6445 - loss: 1.0750
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8708 - department_output_accuracy: 0.6443 - loss: 1.0567
Epoch 8/100
35/35 ━━━━━━━━

In [ ]:
# Evaluate the model with the testing data


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7868 - department_output_accuracy: 0.4877 - loss: 4.0711  


[4.038690090179443, 0.7880434989929199, 0.5]

In [ ]:
# Print the accuracy for both department and attrition


Attrition predictions accuracy: 0.7880434989929199
Department predictions accuracy: 0.5


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.